In [21]:
%matplotlib inline
import torch
from IPython import display
from matplotlib import pyplot as plt
import numpy as np
import random

In [22]:
num_inputs = 2
num_examples = 1000
true_w = [2, -3.4]
true_b = 4.2
features = torch.tensor(np.random.normal(0, 1, (num_examples, num_inputs)), dtype=torch.float)
labels = true_w[0] * features[:, 0] + true_w[1] * features[:, 1] + true_b
labels += torch.tensor(np.random.normal(0, 0.01, size=labels.size()), dtype=torch.float)


In [23]:
import torch.utils.data as Data

batch_size = 10
# 将训练数据的特征和标签组合
dataset = Data.TensorDataset(features, labels)
# 随机读取小批量
data_iter = Data.DataLoader(dataset, batch_size, shuffle=True)


In [24]:
for X, y in data_iter:
    print(X, y)
    break

tensor([[ 0.1471,  1.6050],
        [-2.9934,  1.1167],
        [-1.0160,  1.5409],
        [-0.7699, -0.2495],
        [-1.9546, -0.7983],
        [-0.6526,  0.0904],
        [-0.1811,  0.7229],
        [ 0.6465,  0.3165],
        [ 0.6542,  0.1741],
        [-1.4266, -1.3222]])tensor([-0.9590, -5.5744, -3.0583,  3.5143,  2.9975,  2.5830,  1.3734,  4.4053,
         4.9084,  5.8411])


In [25]:
import torch.nn as nn
class LinearNet(nn.Module): ##继承nn.Module类
    def __init__(self, n_feature):
        super(LinearNet, self).__init__() ##使用父类进行初始化
        self.linear = nn.Linear(n_feature, 1)
    # forward 定义前向传播
    def forward(self, x):
        y = self.linear(x)
        return y

net = LinearNet(num_inputs)
print(net) # 使用print可以打印出网络的结构

LinearNet(
  (linear): Linear(in_features=2, out_features=1, bias=True)
)


In [26]:
net = nn.Sequential()
net.add_module('linear', nn.Linear(num_inputs, 1))
print(net)
print(net[0])

Sequential(
  (linear): Linear(in_features=2, out_features=1, bias=True)
)
Linear(in_features=2, out_features=1, bias=True)


In [27]:
for param in net.parameters():
    print(param)

Parameter containing:
tensor([[-0.2744, -0.0168]], requires_grad=True)
Parameter containing:
tensor([-0.1291], requires_grad=True)


In [28]:
from torch.nn import init

init.normal_(net[0].weight, mean=0, std=0.01)
init.constant_(net[0].bias, val=0)  # 也可以直接修改bias的data: net[0].bias.data.fill_(0)


Parameter containing:
tensor([0.], requires_grad=True)

In [29]:
loss = nn.MSELoss()

In [30]:
import torch.optim as optim

optimizer = optim.SGD(net.parameters(), lr=0.03)
print(optimizer)


SGD (
Parameter Group 0
    dampening: 0
    lr: 0.03
    momentum: 0
    nesterov: False
    weight_decay: 0
)


In [31]:
# 调整学习率
for param_group in optimizer.param_groups:
    param_group['lr'] *= 0.1 # 学习率为之前的0.1倍
print(optimizer)

SGD (
Parameter Group 0
    dampening: 0
    lr: 0.003
    momentum: 0
    nesterov: False
    weight_decay: 0
)


In [32]:
num_epochs = 3
for epoch in range(1, num_epochs + 1):
    for X, y in data_iter:
        output = net(X)
        l = loss(output, y.view(-1, 1))
        optimizer.zero_grad() # 梯度清零，等价于net.zero_grad()
        l.backward()
        optimizer.step()
    print('epoch %d, loss: %f' % (epoch, l.item()))

epoch 1, loss: 11.549944
epoch 2, loss: 2.734591
epoch 3, loss: 1.375861


In [33]:
dense = net[0]
print(true_w, dense.weight)
print(true_b, dense.bias)

[2, -3.4]Parameter containing:
tensor([[ 1.7204, -2.7497]], requires_grad=True)
4.2Parameter containing:
tensor([3.5031], requires_grad=True)
